In [1]:
import socketserver
import json
import datetime
import os
import sys
import time
import threading
import socket
from pathlib import Path
import paho.mqtt.client as mqtt_client
import logging
import configparser
from logging.handlers import TimedRotatingFileHandler

ROOT_DIR = os.path.dirname(os.path.abspath(__vsc_ipynb_file__))
CONFIG = Path(ROOT_DIR) / 'nodecommsrv.ini'

file_encoding = 'utf-8'
recv_encoding = 'CP949'
def create_rotating_log(path,_config):
    _logger_level = _config['LOG_LEVEL']
    _logger_when = _config['LOG_WHEN']
    _logger_interval = _config['LOG_INTERVAL']
    _logger_backupcount = _config['LOG_BACKUPCOUNT']



if __name__ == "__main__":
    
    _config = configparser.ConfigParser()
    _config.read(CONFIG, encoding=file_encoding)

    _command_file_path = _config['APP']['CMD_FILE']

    _host = _config['SOCKET']['SERVER_HOST1']
    _port = int(_config['SOCKET']['SERVER_PORT1'])

    _mqhost = _config['MQTT']['BROKER_HOST'] # Server IP
    _mqport = int(_config['MQTT']['BROKER_PORT']) # Server Port

    _subscribes = _config['MQTT']['TOPIC_SUBS'] # Topic subscribes
    subcribes = _subscribes.split(',')

    _topic_subs_base = _config['MQTT']['TOPIC_SUBS_BASE']

    _pub_init_topic = _config['MQTT']['PUB_INIT_TOPIC']
    _pub_edgenode_topic = _config['MQTT']['PUB_EDGENODE_TOPIC']

    _edgeid = _config['APP']['EDGEID']
    _edgety = _config['APP']['EDGETY']

    HOST, PORT = _host, _port

    # 포트 5000에서 localhost에 바인딩하여 서버 오픈
    with socketserver.TCPServer((HOST, PORT), MyTCPHandler) as server:
        # 서버를 활성화, 실행이 유지됨.
        # Ctrl-C로 프로그램을 중단
        server.serve_forever()

    with open(_command_file_path, 'r', encoding='utf-8') as file:
        command_tbl = json.load(file)
    full_path = os.path.join(ROOT_DIR,"logs","nodecommsrv.log")
    create_rotating_log(full_path, _config['LOGGER'])


NameError: name 'MyTCPHandler' is not defined

In [ ]:
import socketserver
import logging
import datetime
import configparser
from pathlib import Path
import os
from logging.handlers import TimedRotatingFileHandler
import json
import socket
import sys
import threading

client_sockets_1 =[]
ROOT_DIR = os.path.dirname(os.path.abspath(__vsc_ipynb_file__))
CONFIG = Path(ROOT_DIR) / 'nodecommsrv.ini'
logger = None
file_encoding = 'utf-8'
recv_encoding = 'utf-8'


def create_rotating_log(path, _config):

    _logger_level = _config['LOG_LEVEL']
    _logger_when = _config['LOG_WHEN']
    _logger_interval = _config['LOG_INTERVAL']
    _logger_backupcount = _config['LOG_BACKUPCOUNT']

    set_level = logging.INFO

    if _logger_level == "INFO":
        set_level = logging.INFO
    elif _logger_level == "DEBUG":
        set_level = logging.DEBUG
    elif _logger_level == "NOTICE":
        set_level = logging.NOTICE
    elif _logger_level == "ERR":
        set_level = logging.ERR

    global logger

    logging.basicConfig(level=set_level,format='%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] %(message)s')
    logger = logging.getLogger("")
    logger.setLevel(set_level)

    handler = TimedRotatingFileHandler(path,
                                       when=_logger_when,
                                       interval=int(_logger_interval),
                                       backupCount=int(_logger_backupcount),
                                       encoding='utf-8')

    handler.suffix = "-%Y%m%d_%H-%M-%S"

    formatter = logging.Formatter('%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] %(message)s' )
    handler.setFormatter(formatter)

    logger.addHandler(handler)


def get_log():
    return logger


class MyTCPHandler(socketserver.BaseRequestHandler):
    logger = get_log()
    def __init__(self,request,client_address,server):
        logging.info("__init__ 호출 MyTCPHandler")
        socketserver.BaseRequestHandler.__init__(self, request, client_address,server)

        return

    def setup(self):
        logging.info("특장차 setup 호출")
        return socketserver.BaseRequestHandler.setup(self)

    def handle(self):
        buf = ""
        # self.request는 클라이언트에 연결된 TCP 소켓
        data = self.request.recv(4048).decode(encoding='utf-8')

        buf += data
        data = json.loads(data)

        os.makedirs("output",exist_ok=True)

        output_file = os.path.join('output',"tractor_info.log")
        with open(output_file,'a',encoding='utf-8') as of:
            for i in data:
                json.dump(data,of)
                of.write('\n')
        print(data)
        
class ThreadedTCPRequestHandler(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass

if __name__ == "__main__":

    _config = configparser.ConfigParser()
    _config.read(CONFIG, encoding=file_encoding)

    _command_file_path = _config['APP']['CMD_FILE']

    _host = _config['SOCKET']['SERVER_HOST1']
    _port = int(_config['SOCKET']['SERVER_PORT1'])

    _mqhost = _config['MQTT']['BROKER_HOST'] # Server IP
    _mqport = int(_config['MQTT']['BROKER_PORT']) # Server Port

    _subscribes = _config['MQTT']['TOPIC_SUBS'] # Topic subscribes
    subcribes = _subscribes.split(',')

    _topic_subs_base = _config['MQTT']['TOPIC_SUBS_BASE']

    _pub_init_topic = _config['MQTT']['PUB_INIT_TOPIC']
    _pub_edgenode_topic = _config['MQTT']['PUB_EDGENODE_TOPIC']

    _edgeid = _config['APP']['EDGEID']
    _edgety = _config['APP']['EDGETY']

    HOST, PORT = _host, _port

    try:
        server = ThreadedTCPRequestHandler((socket.gethostbyname(HOST),PORT),MyTCPHandler)
    except ValueError as e:
        print(e)
        sys.exit()
    except:
        print("bind failed Closing...")
        sys.exit()
        
    server_thread = threading.Thread(target=server.serve_forever)
    server_thread.daemon = True
    server_thread.start()
    
    try:
        server.serve_forever()
    except KeyboardInterrupt as e:
        print(e)
    except OSError:
        print("Main OSError")

    with open(_command_file_path, 'r', encoding='utf-8') as file:
        command_tbl = json.load(file)
    full_path = os.path.join(ROOT_DIR,"logs","nodecommsrv.log")
    create_rotating_log(full_path, _config['LOGGER'])

2024-11-04 13:54:10,703 - INFO - [355974344.py:66] __init__ 호출 MyTCPHandler
2024-11-04 13:54:10,708 - INFO - [355974344.py:72] 특장차 setup 호출


{'command': '80004', 'type': 'AUTONOMOUS_ELECTRIC_STREET TRAFFIC_INFO DATA', 'VID': 'TRAC000001', 'edgeTy': 'CLS', 'CRC': '12345678', 'timestamp': '2024-11-04 13:54:10.742641', 'data': [{'trafficLightId': '1001', 'trafficLight': '5', 'color': '0', 'shape': '1', 'status': '1'}]}
